In [1]:
from enum import Enum
from collections import defaultdict
import pprint as pp

1. Implement the lion and cow example on a 10x10 grid (the lion's 
starting position is at the bottom left square, and the cow is in the 
top right square). 

In [2]:
class Action(Enum):
    STAY = 0
    NORTH = 1
    SOUTH = 2
    EAST = 3
    WEST = 4

In [3]:
reward = {
    (1, Action.WEST) : 0,
    (2, Action.WEST) : 0,
    (2, Action.NORTH) : 0,
    (2, Action.EAST) : 0,
    (3, Action.NORTH) : 50,
    (4, Action.NORTH) : 50,
    (5, Action.NORTH) : 0,
    (5, Action.EAST) : 0,
    (6, Action.STAY) : 0,
    (7, Action.EAST) : 100
}

transition = {
    (1, Action.WEST) : 2,
    (2, Action.WEST) : 3,
    (2, Action.NORTH) : 4,
    (2, Action.EAST) : 1,
    (3, Action.NORTH) : 5,
    (4, Action.NORTH) : 6,
    (5, Action.NORTH) : 7,
    (5, Action.EAST) : 4,
    (6, Action.STAY) : 6,
    (7, Action.EAST) : 6
}

state_reward = defaultdict(int)
state_reward[5] = 50
state_reward[6] = 100

N_STATES = 7

In [4]:
states = list(range(1, 8))
states

[1, 2, 3, 4, 5, 6, 7]

In [5]:
actions = {
    1: [Action.WEST],
    2: [Action.WEST, Action.NORTH, Action.EAST],
    3: [Action.NORTH],
    4: [Action.NORTH],
    5: [Action.NORTH, Action.EAST],
    6: [Action.STAY],
    7: [Action.EAST]
}

### a) Compute the V* values for each state with discount factor gamma = 0.8. 

In [6]:
def value_iteration(gamma, theta):
    values = {}

    for s in states:
        values[s] = 0
    niter = 0
    while True:
        niter += 1
        delta = 0
        for state in states:
            v = values[state]
            candidates = [r + gamma * values[ss]
                          for a in actions[state] 
                          for ss in [transition[(state, a)]] for r in [reward[(state, a)]]]
            values[state] = max(candidates)
            delta = max(delta, abs(v - values[state]))
        if delta < theta:
            return values

In [7]:
values = value_iteration(gamma=0.8, theta=1)

In [8]:
print('{0: <10} {1: <6}'.format('state', 'V*'))
for s in states:
    print("{0: <10} {1: <5}".format(s, values[s]))

state      V*    
1          72.96000000000001
2          91.2 
3          114.0
4          50.0 
5          80.0 
6          0.0  
7          100.0


### b) What is the optimal policy when gamma = 0.8?

In [9]:
def get_policy(values):
    policy = {}
    for s in states:
        pairs = [(values[transition[s,a]], a) for a in actions[s]]
        policy[s] = sorted(pairs)[0][1]
        
    return policy

In [10]:
values = value_iteration(gamma=0.8, theta=1)
get_policy(values)

{1: <Action.WEST: 4>,
 2: <Action.NORTH: 1>,
 3: <Action.NORTH: 1>,
 4: <Action.NORTH: 1>,
 5: <Action.EAST: 3>,
 6: <Action.STAY: 0>,
 7: <Action.EAST: 3>}

### c) Does the optimal policy change if gamma is set to 0.5 instead? If yes, give the new policy. If not, explain. 

In [11]:
values = value_iteration(gamma=0.5, theta=1)
get_policy(values)

{1: <Action.WEST: 4>,
 2: <Action.EAST: 3>,
 3: <Action.NORTH: 1>,
 4: <Action.NORTH: 1>,
 5: <Action.EAST: 3>,
 6: <Action.STAY: 0>,
 7: <Action.EAST: 3>}

### d) Compute the Q(s,a) values for the following state action pairs: (S2,West), (S6,Stay), (S3, North). Let gamma = 0.8 and alpha = 1.

In [17]:
def Q_model_based(V_star, state, action, gamma, reward=reward, transition=transition):
    return reward[state, action] + gamma * V_star[transition[state, action]]

In [18]:
values = value_iteration(gamma=0.8, theta=1)
print('{: <10} {: <15}'.format('state', 'action'))
for s in states:
    print("{0: <10} {1: <5}".format(s, values[s]))

state      action         
1          72.96000000000001
2          91.2 
3          114.0
4          50.0 
5          80.0 
6          0.0  
7          100.0


In [20]:
print('{: <10} {: <15} {: <3}'.format('state', 'action', 'Q'))

for state, act in [(2, Action.WEST), (6, Action.STAY), (3, Action.NORTH)]:
    print('{: <10} {: <15} {}'.format(state, act, Q_model_based(values, state, act, gamma=0.8)))

state      action          Q  
2          Action.WEST     91.2
6          Action.STAY     0.0
3          Action.NORTH    114.0


### e) Consider applying the Q-learning algorithm to the "treasure-hunting" game. Let Q' be the estimate of Q. Initially all Q' values are set to 0, and gamma = 0.8 and alpha = 1. Assume that the agent moves from state S1, via states S2, S3, S5, and S7, to state S6. Show how the Q' values are updated during this episode. Repeat the same episode twice more and show how the Q' values are revised during each episode.

In [68]:
def init_QQ():
    QQ = {}
    for state in states:
        for act in actions[state]:
            QQ[(state, act)] = 0
    return QQ

In [138]:
def one_run(QQ, s, moves, gamma=0.8):
    for move in moves:
        print("s", s)
        r = reward[(s, move)]
        print('action {}'.format(move))
        print('reward: {}'.format(r))
        ss = transition[s, move]
        print('new_state: {}'.format(ss))
        QQ[(s, move)] = r + gamma * max([QQ[(ss, a)] for a in actions[ss]])
        pp.PrettyPrinter(indent=4).pprint(QQ)
        print()
        s = ss
        
    return QQ, s

In [139]:
QQ = init_QQ()
start_state = 1
moves = [Action.WEST, Action.WEST, Action.NORTH, Action.NORTH, Action.EAST]

In [141]:
QQ2, s2 = one_run(QQ, start_state, moves)

s 1
action Action.WEST
reward: 0
new_state: 2
{   (1, <Action.WEST: 4>): 0.0,
    (2, <Action.EAST: 3>): 0,
    (2, <Action.NORTH: 1>): 0,
    (2, <Action.WEST: 4>): 0,
    (3, <Action.NORTH: 1>): 0,
    (4, <Action.NORTH: 1>): 0,
    (5, <Action.NORTH: 1>): 0,
    (5, <Action.EAST: 3>): 0,
    (6, <Action.STAY: 0>): 0,
    (7, <Action.EAST: 3>): 0}

s 2
action Action.WEST
reward: 0
new_state: 3
{   (1, <Action.WEST: 4>): 0.0,
    (2, <Action.EAST: 3>): 0,
    (2, <Action.NORTH: 1>): 0,
    (2, <Action.WEST: 4>): 0.0,
    (3, <Action.NORTH: 1>): 0,
    (4, <Action.NORTH: 1>): 0,
    (5, <Action.NORTH: 1>): 0,
    (5, <Action.EAST: 3>): 0,
    (6, <Action.STAY: 0>): 0,
    (7, <Action.EAST: 3>): 0}

s 3
action Action.NORTH
reward: 50
new_state: 5
{   (1, <Action.WEST: 4>): 0.0,
    (2, <Action.EAST: 3>): 0,
    (2, <Action.NORTH: 1>): 0,
    (2, <Action.WEST: 4>): 0.0,
    (3, <Action.NORTH: 1>): 50.0,
    (4, <Action.NORTH: 1>): 0,
    (5, <Action.NORTH: 1>): 0,
    (5, <Action.EAST: 3

In [143]:
QQ3, s3 = one_run(QQ2, start_state, moves)

s 1
action Action.WEST
reward: 0
new_state: 2
{   (1, <Action.WEST: 4>): 0.0,
    (2, <Action.EAST: 3>): 0,
    (2, <Action.NORTH: 1>): 0,
    (2, <Action.WEST: 4>): 0.0,
    (3, <Action.NORTH: 1>): 50.0,
    (4, <Action.NORTH: 1>): 0,
    (5, <Action.NORTH: 1>): 0.0,
    (5, <Action.EAST: 3>): 0,
    (6, <Action.STAY: 0>): 0,
    (7, <Action.EAST: 3>): 100.0}

s 2
action Action.WEST
reward: 0
new_state: 3
{   (1, <Action.WEST: 4>): 0.0,
    (2, <Action.EAST: 3>): 0,
    (2, <Action.NORTH: 1>): 0,
    (2, <Action.WEST: 4>): 40.0,
    (3, <Action.NORTH: 1>): 50.0,
    (4, <Action.NORTH: 1>): 0,
    (5, <Action.NORTH: 1>): 0.0,
    (5, <Action.EAST: 3>): 0,
    (6, <Action.STAY: 0>): 0,
    (7, <Action.EAST: 3>): 100.0}

s 3
action Action.NORTH
reward: 50
new_state: 5
{   (1, <Action.WEST: 4>): 0.0,
    (2, <Action.EAST: 3>): 0,
    (2, <Action.NORTH: 1>): 0,
    (2, <Action.WEST: 4>): 40.0,
    (3, <Action.NORTH: 1>): 50.0,
    (4, <Action.NORTH: 1>): 0,
    (5, <Action.NORTH: 1>): 0.0,